<a href="https://colab.research.google.com/github/mani2106/Competition-Notebooks/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connecting and importing data

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!apt-get install build-essential swig
!pip install auto-sklearn

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
swig is already the newest version (3.0.12-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [0]:
import autosklearn.classification
import pandas as pd
import os
import featuretools as ft

path = 'gdrive/My Drive/Comp data'

In [0]:
train_set = pd.read_csv(os.path.join(path, 'feature_agg.csv'))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,4,180,181,182,183,219,220,259,260,261) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
train_labels = pd.read_csv(os.path.join(path, 'train_foreclosure.csv'))
test_labels = pd.read_csv(os.path.join(path, 'test_foreclosure.csv'))

In [0]:
train_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20012 entries, 0 to 20011
Data columns (total 2 columns):
AGREEMENTID    20012 non-null int64
FORECLOSURE    20012 non-null int64
dtypes: int64(2)
memory usage: 312.8 KB


In [0]:
train_set.iloc[:, [3,4,180,181,182,183,219,220,259,260,261]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32895 entries, 0 to 32894
Data columns (total 11 columns):
NPA_IN_LAST_MONTH                    843 non-null object
NPA_IN_CURRENT_MONTH                 843 non-null object
customers.SEX                        9606 non-null object
customers.MARITAL_STATUS             9604 non-null object
customers.QUALIFICATION              9397 non-null object
customers.POSITION                   661 non-null object
customers.MODE(Loan data.CITY)       9998 non-null object
customers.MODE(Loan data.PRODUCT)    10000 non-null object
customers.MODE(Email.SubType)        1810 non-null object
customers.MODE(Email.Status)         1810 non-null object
customers.MODE(Email.Type)           1810 non-null object
dtypes: object(11)
memory usage: 2.8+ MB


In [0]:
bool_cols = ["NPA_IN_LAST_MONTH", "NPA_IN_CURRENT_MONTH",
             "customers.MARITAL_STATUS", "customers.SEX"]

cat_cols = ["customers.POSITION", "customers.QUALIFICATION",
           "customers.MODE(Loan data.CITY)", 
           "customers.MODE(Loan data.PRODUCT)",
           "customers.MODE(Email.SubType)", "customers.MODE(Email.Status)",
           "customers.MODE(Email.Type)"]

train_set[bool_cols] = train_set[bool_cols].astype('bool')

In [0]:
for col in cat_cols:
  train_set[col] = train_set[col].astype('category')

In [0]:
data_dummied = pd.get_dummies(train_set)

In [0]:
train_data = data_dummied[data_dummied['AGREEMENTID'].isin(train_labels['AGREEMENTID'].tolist())]
test_data = data_dummied[data_dummied['AGREEMENTID'].isin(test_labels['AGREEMENTID'].tolist())]

In [0]:
print(train_data.shape,
test_data.shape,
data_dummied.shape)

(19731, 974) (13164, 974) (32895, 974)


In [0]:
import sklearn.model_selection
import sklearn.metrics

X_train, X_test, y_train, y_test = \
        sklearn.model_selection.train_test_split(
    train_data,
    # only the train labels in the available data
    train_labels[train_labels['AGREEMENTID'].isin(train_data['AGREEMENTID'].tolist())]['FORECLOSURE'],
    random_state=1
)

In [0]:
automl = autosklearn.classification.AutoSklearnClassifier(
        tmp_folder='/tmp/autosklearn_cv_example_tmp',
        output_folder='/tmp/autosklearn_cv_example_out',
        delete_tmp_folder_after_terminate=False,
        resampling_strategy='cv',
        resampling_strategy_arguments={'folds': 3},
    )

# fit() changes the data in place, but refit needs the original data. We
# therefore copy the data. In practice, one should reload the data
automl.fit(X_train.copy(), y_train.copy(), dataset_name='cust_loan')
# During fit(), models are fit on individual cross-validation folds. To use
# all available data, we call refit() which trains all models in the
# final ensemble on the whole dataset.
automl.refit(X_train.copy(), y_train.copy())

print(automl.show_models())

predictions = automl.predict(X_test)
print("Accuracy score", sklearn.metrics.accuracy_score(y_test, predictions))


In [0]:
fore_predictions = automl.predict(test_data)

# test_labels['FORECLOSURE'] = fore_predictions

# print(len(fore_predictions), test_labels.shape)

test_labels['FORECLOSURE'] = pd.Series(fore_predictions)

In [0]:
test_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13342 entries, 0 to 13341
Data columns (total 2 columns):
AGREEMENTID    13342 non-null int64
FORECLOSURE    13342 non-null float64
dtypes: float64(1), int64(1)
memory usage: 208.5 KB


In [0]:
test_labels.fillna(method = 'ffill', inplace = True)

In [0]:
test_labels.to_csv(os.path.join(path, 'predictions.csv'), index = False)